In [ ]:
#Autoencoder + LogisticHazard

In [ ]:
! pip install sklearn-pandas
! pip install pycox
! pip install lifelines

     |████████████████████████████████| 81kB 6.2MB/s 
     |████████████████████████████████| 51kB 5.7MB/s 
     |████████████████████████████████| 348kB 7.8MB/s 
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-cp36-none-any.whl size=4035 sha256=1392b4dd374d6f3602b438109a82c8105a88e0ce3ee0c8c369b92565e0a000dd
  Stored in directory: /root/.cache/pip/wheels/dc/68/dc/91321c55fba449755524481854f5be70d41912b8f886f908bb
Successfully built autograd-gamma


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# For preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper 


import torch # For building the networks 
from torch import nn
import torch.nn.functional as F
import torchtuples as tt # Some useful functions

from pycox.datasets import metabric
from pycox.models import LogisticHazard
from pycox.models.loss import NLLLogistiHazardLoss, CoxPHLoss, NLLPMFLoss,NLLPCHazardLoss
from pycox.evaluation import EvalSurv

from lifelines.utils import concordance_index
from lifelines import CoxPHFitter
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

np.random.seed(1234)
_ = torch.manual_seed(1234)

In [ ]:

main_folder = '/content/drive/My Drive/deepsurv_DTF/' #change it to the path to deepsurv directory

# import the top 10 ranks selected by Lasso COX model on factorized data ranks range from 2 to 200

#from numpy import genfromtxt
ranklist = np.genfromtxt(main_folder + 'topranklist.csv', delimiter=',')
ranklist = np.array(ranklist, dtype=np.int)
ranklist = ranklist[0:10]

# ranks from 10 to 200 by step size 10
#ranklist = np.arange(10,201,20)

fac_folder = main_folder + 'allfacdata/'
ind_folder = main_folder + 'split_ind/'
data_folder = main_folder + 'clinical_data/'
result_folder = main_folder + 'result/'

gedata_folder = main_folder + 'datasets/'

clinical = pd.read_csv(data_folder + "clinical_10y.csv") #read in clinical data

Y = np.array(clinical["OS.time"])
E = np.array(clinical["OS"])
Age = np.array(clinical["agegroup"])
Stage = np.array(clinical["stagegroup"])


In [ ]:
ranklist = [17] #set rank for CP-ALS

In [ ]:

resultdf = []
for ind in np.arange(1,21): #go through all 20 train/test pairs

  
  for i in ranklist:
    np.random.seed(1234)
    _ = torch.manual_seed(1234)
    print('seed:' + str(ind))
    print('rank: ' + str(i))

    df_train = pd.read_csv(fac_folder + 'train' + str(ind) + "rank" + str(i) + ".csv",header = None)
    df_test = pd.read_csv(fac_folder + 'test' + str(ind) + "rank"+ str(i) + ".csv",header= None)
    testind = np.array(pd.read_csv(ind_folder + 'testind' + str(ind)+ ".csv")["x"]) - 1 # minus 1 to match the array index in Python
    trainind = np.array(pd.read_csv(ind_folder + 'trainind' + str(ind)+ ".csv")["x"]) - 1

    df_train["age"] = Age[trainind]
    df_train["stage"] = Stage[trainind]
    df_test["age"] = Age[testind]
    df_test["stage"] = Stage[testind]

    Y_train = Y[trainind]
    Y_test = Y[testind]
    E_train = E[trainind]
    E_test = E[testind]

    df_train["time"] = Y_train.astype('float32')
    df_train["event"] = E_train.astype('float32')
    df_train,df_val = train_test_split(df_train, test_size=0.2)
    df_test["time"] = Y_test.astype('float32')
    df_test["event"] = E_test.astype('float32')

    #cols_standardize = ['x0', 'x1', 'x2', 'x3', 'x8']
    var_num = i + 2
    cols_leave = df_train.columns[0:var_num]
    print(cols_leave)
    #standardize = [([col], StandardScaler()) for col in cols_standardize]
    leave = [(col, None) for col in cols_leave]
    x_mapper = DataFrameMapper(leave)
    x_train = x_mapper.fit_transform(df_train).astype('float32')
    x_val = x_mapper.transform(df_val).astype('float32')
    x_test = x_mapper.transform(df_test).astype('float32')


    num_durations = 100
    labtrans = LogisticHazard.label_transform(num_durations)
    get_target = lambda df: (df['time'].values, df['event'].values)
    y_train_surv = labtrans.fit_transform(*get_target(df_train))
    y_val_surv = labtrans.transform(*get_target(df_val))

    train = tt.tuplefy(x_train, (y_train_surv, x_train))
    val = tt.tuplefy(x_val, (y_val_surv, x_val))

    # We don't need to transform the test labels
    durations_test, events_test = get_target(df_test)
    durations_train, events_train = get_target(df_train)
    
    class NetAESurv(nn.Module):
      def __init__(self, in_features, encoded_features, out_features):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(in_features, 512), nn.ReLU(),
            nn.Linear(512, 128), nn.ReLU(),

            nn.Linear(128, encoded_features),
        )
        self.decoder = nn.Sequential(
            nn.Linear(encoded_features, 128), nn.ReLU(),
    
  
            nn.Linear(128, 512), nn.ReLU(),
            nn.Linear(512, in_features),
        )
        self.surv_net = nn.Sequential(
            nn.Linear(encoded_features, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
           # torch.nn.BatchNorm1d(32),
            #torch.nn.BatchNorm1d(16),
            nn.Linear(16, out_features),
        )

      def forward(self, input):
        encoded = self.encoder(input)
        decoded = self.decoder(encoded)
        phi = self.surv_net(encoded)
        return phi, decoded

      def predict(self, input):
        # Will be used by model.predict later.
        # As this only has the survival output, 
        # we don't have to change LogisticHazard.
        encoded = self.encoder(input)
        return self.surv_net(encoded)

    in_features = x_train.shape[1]
    encoded_features = 16
    out_features = labtrans.out_features
    net = NetAESurv(in_features, encoded_features, out_features)

    class LossAELogHaz(nn.Module):
      def __init__(self, alpha):
        super().__init__()
        assert (alpha >= 0) and (alpha <= 1), 'Need `alpha` in [0, 1].'
        self.alpha = alpha
        self.loss_surv = NLLLogistiHazardLoss() #NLLPMFLoss() #
        self.loss_ae = nn.MSELoss()
        
      def forward(self, phi, decoded, target_loghaz, target_ae):
        idx_durations, events = target_loghaz
        loss_surv = self.loss_surv(phi, idx_durations, events)
        loss_ae = self.loss_ae(decoded, target_ae)
        return self.alpha * loss_surv + (1 - self.alpha) * loss_ae

    loss = LossAELogHaz(0.4)
    model = LogisticHazard(net, tt.optim.Adam(0.01), duration_index=labtrans.cuts, loss=loss)

    metrics = dict(
    loss_surv = LossAELogHaz(1),
    loss_ae   = LossAELogHaz(0)
    )
    callbacks = [tt.cb.EarlyStopping()]

    batch_size = 256
    epochs = 500
    model.fit(*train, batch_size, epochs, callbacks, val_data=val, metrics=metrics)


    #training c-index
    surv = model.predict_surv_df(x_train)
    ev = EvalSurv(surv, durations_train, events_train, censor_surv='km')
    trainci =  ev.concordance_td('antolini')
    print(trainci)
    #testing c-index
    surv = model.predict_surv_df(x_test)
    ev = EvalSurv(surv, durations_test, events_test, censor_surv='km')
    testci = ev.concordance_td('antolini')

    print(testci)

    resultdf.append((ind,i,
                      trainci,
                    testci))


  
    cols=['seed','rank','trainCI','testCI']

    results = pd.DataFrame(resultdf,columns = cols)
    avgtestci = results['testCI'].mean()
    avgtrainci = results['trainCI'].mean()
 
    print( "avgTRAIN"+ str(avgtrainci))
    print("avgTEST" + str(avgtestci))
avglist = []

resultname = result_folder + '12.3.auto_loghazard' + 'rank17' + ".csv"
results.to_csv(resultname,index=False,header = True)


seed:1
rank: 17
Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 'age',
       'stage'],
      dtype='object')
0:	[0s / 0s],		train_loss: 9.0792,	train_loss_surv: 22.5882,	train_loss_ae: 0.0731,	val_loss: 6.8748,	val_loss_surv: 17.1049,	val_loss_ae: 0.0548
1:	[0s / 0s],		train_loss: 7.8509,	train_loss_surv: 19.5702,	train_loss_ae: 0.0380,	val_loss: 4.6228,	val_loss_surv: 11.4875,	val_loss_ae: 0.0463
2:	[0s / 0s],		train_loss: 5.2311,	train_loss_surv: 13.0104,	train_loss_ae: 0.0449,	val_loss: 3.4386,	val_loss_surv: 8.5338,	val_loss_ae: 0.0418
3:	[0s / 0s],		train_loss: 3.9237,	train_loss_surv: 9.7603,	train_loss_ae: 0.0326,	val_loss: 2.1465,	val_loss_surv: 5.3387,	val_loss_ae: 0.0183
4:	[0s / 0s],		train_loss: 2.4640,	train_loss_surv: 6.1327,	train_loss_ae: 0.0181,	val_loss: 1.5657,	val_loss_surv: 3.8925,	val_loss_ae: 0.0144
5:	[0s / 0s],		train_loss: 1.7933,	train_loss_surv: 4.4603,	train_loss_ae: 0.0152,	val_loss: 1.0108,	val_loss_surv: 2.5068,	val_loss_ae: 0.0134
6:	[

In [ ]:

resultname = result_folder + '11.18.auto_deepsurv' + 'rank10-198' + ".csv"
results.to_csv(resultname,index=False,header = True)

In [ ]:
resultname

'/content/drive/My Drive/deepsurv_DTF/result/auto_deepsurvrank10-198.11.18.csv'

In [ ]:
len(np.arange(10,200,4))*20

960

In [ ]:
#compare

In [ ]:
lossweight = [0.3, 0.5, 0.7]
encodenum = [10, 20, 30]
netsize = [16, 32, 64]
encodesize = [ 32, 64, 128]
list15_23 = np.arange(16,19)

In [ ]:
avglist = []
resultdf = []


for ind in np.arange(1,21): #go through all 20 train/test pairs

  for lw in lossweight:
    for en in encodenum:
      for ns in netsize:
        for es in encodesize:
          for i in list15_23:
            
            print('seed:' + str(ind))
            print('rank: ' + str(i))

            df_train = pd.read_csv(fac_folder + 'train' + str(ind) + "rank" + str(i) + ".csv",header = None)
            df_test = pd.read_csv(fac_folder + 'test' + str(ind) + "rank"+ str(i) + ".csv",header= None)
            testind = np.array(pd.read_csv(ind_folder + 'testind' + str(ind)+ ".csv")["x"]) - 1 # minus 1 to match the array index in Python
            trainind = np.array(pd.read_csv(ind_folder + 'trainind' + str(ind)+ ".csv")["x"]) - 1

            df_train["age"] = Age[trainind]
            df_train["stage"] = Stage[trainind]
            df_test["age"] = Age[testind]
            df_test["stage"] = Stage[testind]

            Y_train = Y[trainind]
            Y_test = Y[testind]
            E_train = E[trainind]
            E_test = E[testind]

            df_train["time"] = Y_train.astype('float32')
            df_train["event"] = E_train.astype('float32')
            df_train,df_val = train_test_split(df_train, test_size=0.2)
            df_test["time"] = Y_test.astype('float32')
            df_test["event"] = E_test.astype('float32')

            #cols_standardize = ['x0', 'x1', 'x2', 'x3', 'x8']
            var_num = i + 2
            cols_leave = df_train.columns[0:var_num]
            print(cols_leave)
            #standardize = [([col], StandardScaler()) for col in cols_standardize]
            leave = [(col, None) for col in cols_leave]
            x_mapper = DataFrameMapper(leave)
            x_train = x_mapper.fit_transform(df_train).astype('float32')
            x_val = x_mapper.transform(df_val).astype('float32')
            x_test = x_mapper.transform(df_test).astype('float32')


            num_durations = 100
            labtrans = LogisticHazard.label_transform(num_durations)
            get_target = lambda df: (df['time'].values, df['event'].values)
            y_train_surv = labtrans.fit_transform(*get_target(df_train))
            y_val_surv = labtrans.transform(*get_target(df_val))

            train = tt.tuplefy(x_train, (y_train_surv, x_train))
            val = tt.tuplefy(x_val, (y_val_surv, x_val))

            # We don't need to transform the test labels
            durations_test, events_test = get_target(df_test)
            durations_train, events_train = get_target(df_train)
            
            class NetAESurv(nn.Module):
              def __init__(self, in_features, encoded_features, out_features):
                super().__init__()
                self.encoder = nn.Sequential(
                    nn.Linear(in_features, es*4), nn.ReLU(),
                    nn.Linear(es*4, es), nn.ReLU(),
            
                    nn.Linear(es, encoded_features),
                )
                self.decoder = nn.Sequential(
                    nn.Linear(encoded_features, es), nn.ReLU(),
            
                    nn.Linear(es, es*4), nn.ReLU(),
                    nn.Linear(es*4, in_features),
                )
                self.surv_net = nn.Sequential(
                    nn.Linear(encoded_features, ns*3), nn.ReLU(),
                    nn.Linear(ns*3, ns*2), nn.ReLU(),
                    nn.Linear(ns*2, ns), nn.ReLU(),
                    nn.Linear(ns, out_features),
                )

              def forward(self, input):
                encoded = self.encoder(input)
                decoded = self.decoder(encoded)
                phi = self.surv_net(encoded)
                return phi, decoded


              class NetAESurv(nn.Module):
                def __init__(self, in_features, encoded_features, out_features):
                  super().__init__()
                  self.encoder = nn.Sequential(
                      nn.Linear(in_features, 512), nn.ReLU(),
                      nn.Linear(512, 128), nn.ReLU(),
              
                      nn.Linear(128, encoded_features),
                  )
                  self.decoder = nn.Sequential(
                      nn.Linear(encoded_features, 128), nn.ReLU(),
              
                      nn.Linear(128, 512), nn.ReLU(),
                      nn.Linear(512, in_features),
                  )
                  self.surv_net = nn.Sequential(
                      nn.Linear(encoded_features, 32), nn.ReLU(),
                      nn.Linear(32, 16), nn.ReLU(),
                      nn.Linear(16, out_features),
                  )

                def forward(self, input):
                  encoded = self.encoder(input)
                  decoded = self.decoder(encoded)
                  phi = self.surv_net(encoded)
                  return phi, decoded

      def predict(self, input):
        # Will be used by model.predict later.
        # As this only has the survival output, 
        # we don't have to change LogisticHazard.
        encoded = self.encoder(input)
        return self.surv_net(encoded)

              def predict(self, input):
                # Will be used by model.predict later.
                # As this only has the survival output, 
                # we don't have to change LogisticHazard.
                encoded = self.encoder(input)
                return self.surv_net(encoded)

            in_features = x_train.shape[1]
            encoded_features = en
            out_features = labtrans.out_features
            net = NetAESurv(in_features, encoded_features, out_features)

            class LossAELogHaz(nn.Module):
              def __init__(self, alpha):
                super().__init__()
                assert (alpha >= 0) and (alpha <= 1), 'Need `alpha` in [0, 1].'
                self.alpha = alpha
                self.loss_surv = NLLLogistiHazardLoss() #NLLPMFLoss() #
                self.loss_ae = nn.MSELoss()
                
              def forward(self, phi, decoded, target_loghaz, target_ae):
                idx_durations, events = target_loghaz
                loss_surv = self.loss_surv(phi, idx_durations, events)
                loss_ae = self.loss_ae(decoded, target_ae)
                return self.alpha * loss_surv + (1 - self.alpha) * loss_ae

            loss = LossAELogHaz(lw)
            model = LogisticHazard(net, tt.optim.Adam(0.01), duration_index=labtrans.cuts, loss=loss)

            metrics = dict(
            loss_surv = LossAELogHaz(1),
            loss_ae   = LossAELogHaz(0)
            )
            callbacks = [tt.cb.EarlyStopping()]

            batch_size = 16
            epochs = 200
            model.fit(*train, batch_size, epochs, callbacks, val_data=val, metrics=metrics)

            def check_ci(ci):
              if ci < 0.5:
                ci = 1 - ci
              else:
                  ci = ci
              return ci
            #training c-index
            surv = model.predict_surv_df(x_train)
            ev = EvalSurv(surv, durations_train, events_train, censor_surv='km')
            # trainci = check_ci(1- ev.concordance_td('antolini'))
            trainci = check_ci(ev.concordance_td())
            print(trainci)
            #testing c-index
            surv = model.predict_surv_df(x_test)
            ev = EvalSurv(surv, durations_test, events_test, censor_surv='km')
            # testci = check_ci(1- ev.concordance_td('antolini'))
            testci = check_ci(ev.concordance_td())
            print(testci)

            resultdf.append((ind,i,lw,en,es,ns,
                              trainci,
                            testci))


  
  cols=['seed','rank','lossweight','encodeNum','endoeSize','netSize','trainCI','testCI']

  results = pd.DataFrame(resultdf,columns = cols)
  avgtestci = results['testCI'].mean()
  avgtrainci = results['trainCI'].mean()
  print(avgtestci)
  print(avgtrainci)
  #avglist = []

  resultname = result_folder + '11.21.auto_deepsurv' + 'rank15-23compare' + ".csv"
  results.to_csv(resultname,index=False,header = True)
    
#cols=['seed','rank','trainCI','testCI']

#cols1=['run','rank','trainCI','testCI']

#results = pd.DataFrame(resultdf,columns = cols)
#avgtestci = results['testCI'].mean()
#avgtrainci = results['trainCI'].mean()
#resultmean = pd.DataFrame(resultmeandf,columns = cols1)
#print(avgtestci)
#print(avgtrainci)
#resultname = result_folder + 'auto_deepsurv' + 'ranklisttopcox' +'simplenet' + ".csv"
#results.to_csv(resultname,index=False,header = True)


Streaming output truncated to the last 5000 lines.
0.6878471087879778
0.6463490099009901
seed:20
rank: 16
Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 'age', 'stage'], dtype='object')
0:	[0s / 0s],		train_loss: 1.7257,	train_loss_surv: 5.6682,	train_loss_ae: 0.0361,	val_loss: 0.4774,	val_loss_surv: 1.5588,	val_loss_ae: 0.0139
1:	[0s / 0s],		train_loss: 0.2997,	train_loss_surv: 0.9608,	train_loss_ae: 0.0164,	val_loss: 0.3251,	val_loss_surv: 1.0471,	val_loss_ae: 0.0157
2:	[0s / 0s],		train_loss: 0.2212,	train_loss_surv: 0.7080,	train_loss_ae: 0.0126,	val_loss: 0.2909,	val_loss_surv: 0.9534,	val_loss_ae: 0.0070
3:	[0s / 0s],		train_loss: 0.2179,	train_loss_surv: 0.7089,	train_loss_ae: 0.0074,	val_loss: 0.2916,	val_loss_surv: 0.9523,	val_loss_ae: 0.0085
4:	[0s / 0s],		train_loss: 0.2163,	train_loss_surv: 0.7007,	train_loss_ae: 0.0088,	val_loss: 0.2950,	val_loss_surv: 0.9578,	val_loss_ae: 0.0110
5:	[0s / 1s],		train_loss: 0.2219,	train_loss_surv: 0.7244,	train_loss_ae: 0.006

In [ ]:
#finalmodel

In [ ]:
avglist = []
resultdf = []


for ind in np.arange(1,21): #go through all 20 train/test pairs
  np.random.seed(1234)
  _ = torch.manual_seed(1234) 
  lw = 0.3
  en  = 20
  ns = 64
  es  = 16
  i = 17
  print('seed:' + str(ind))
  print('rank: ' + str(i))

  df_train = pd.read_csv(fac_folder + 'train' + str(ind) + "rank" + str(i) + ".csv",header = None)
  df_test = pd.read_csv(fac_folder + 'test' + str(ind) + "rank"+ str(i) + ".csv",header= None)
  testind = np.array(pd.read_csv(ind_folder + 'testind' + str(ind)+ ".csv")["x"]) - 1 # minus 1 to match the array index in Python
  trainind = np.array(pd.read_csv(ind_folder + 'trainind' + str(ind)+ ".csv")["x"]) - 1

  df_train["age"] = Age[trainind]
  df_train["stage"] = Stage[trainind]
  df_test["age"] = Age[testind]
  df_test["stage"] = Stage[testind]

  Y_train = Y[trainind]
  Y_test = Y[testind]
  E_train = E[trainind]
  E_test = E[testind]

  df_train["time"] = Y_train.astype('float32')
  df_train["event"] = E_train.astype('float32')
  df_train,df_val = train_test_split(df_train, test_size=0.2)
  df_test["time"] = Y_test.astype('float32')
  df_test["event"] = E_test.astype('float32')

  #cols_standardize = ['x0', 'x1', 'x2', 'x3', 'x8']
  var_num = i + 2
  cols_leave = df_train.columns[0:var_num]
  print(cols_leave)
  #standardize = [([col], StandardScaler()) for col in cols_standardize]
  leave = [(col, None) for col in cols_leave]
  x_mapper = DataFrameMapper(leave)
  x_train = x_mapper.fit_transform(df_train).astype('float32')
  x_val = x_mapper.transform(df_val).astype('float32')
  x_test = x_mapper.transform(df_test).astype('float32')


  num_durations = 100
  labtrans = LogisticHazard.label_transform(num_durations)
  get_target = lambda df: (df['time'].values, df['event'].values)
  y_train_surv = labtrans.fit_transform(*get_target(df_train))
  y_val_surv = labtrans.transform(*get_target(df_val))

  train = tt.tuplefy(x_train, (y_train_surv, x_train))
  val = tt.tuplefy(x_val, (y_val_surv, x_val))

  # We don't need to transform the test labels
  durations_test, events_test = get_target(df_test)
  durations_train, events_train = get_target(df_train)
  
  class NetAESurv(nn.Module):
    def __init__(self, in_features, encoded_features, out_features):
      super().__init__()
      self.encoder = nn.Sequential(
          nn.Linear(in_features, es*4), nn.ReLU(),
          nn.Linear(es*4, es), nn.ReLU(),
  
          nn.Linear(es, encoded_features),
      )
      self.decoder = nn.Sequential(
          nn.Linear(encoded_features, es), nn.ReLU(),
  
          nn.Linear(es, es*4), nn.ReLU(),
          nn.Linear(es*4, in_features),
      )
      self.surv_net = nn.Sequential(
          nn.Linear(encoded_features, ns*2), nn.ReLU(),
          nn.Linear(ns*2, ns*2), nn.ReLU(),
          nn.Linear(ns*2, ns), nn.ReLU(),
          nn.Linear(ns, out_features),
      )

    def forward(self, input):
      encoded = self.encoder(input)
      decoded = self.decoder(encoded)
      phi = self.surv_net(encoded)
      return phi, decoded

    def predict(self, input):
      # Will be used by model.predict later.
      # As this only has the survival output, 
      # we don't have to change LogisticHazard.
      encoded = self.encoder(input)
      return self.surv_net(encoded)

  in_features = x_train.shape[1]
  encoded_features = en
  out_features = labtrans.out_features
  net = NetAESurv(in_features, encoded_features, out_features)

  class LossAELogHaz(nn.Module):
    def __init__(self, alpha):
      super().__init__()
      assert (alpha >= 0) and (alpha <= 1), 'Need `alpha` in [0, 1].'
      self.alpha = alpha
      self.loss_surv = NLLLogistiHazardLoss() #NLLPMFLoss() #
      self.loss_ae = nn.MSELoss()
      
    def forward(self, phi, decoded, target_loghaz, target_ae):
      idx_durations, events = target_loghaz
      loss_surv = self.loss_surv(phi, idx_durations, events)
      loss_ae = self.loss_ae(decoded, target_ae)
      return self.alpha * loss_surv + (1 - self.alpha) * loss_ae

  loss = LossAELogHaz(lw)
  model = LogisticHazard(net, tt.optim.Adam(0.01), duration_index=labtrans.cuts, loss=loss)

  metrics = dict(
  loss_surv = LossAELogHaz(1),
  loss_ae   = LossAELogHaz(0)
  )
  callbacks = [tt.cb.EarlyStopping()]

  batch_size = 16
  epochs = 200
  model.fit(*train, batch_size, epochs, callbacks, val_data=val, metrics=metrics)

  def check_ci(ci):
    if ci < 0.5:
      ci = 1 - ci
    else:
        ci = ci
    return ci
  #training c-index
  surv = model.predict_surv_df(x_train)
  ev = EvalSurv(surv, durations_train, events_train, censor_surv='km')
  # trainci = check_ci(1- ev.concordance_td('antolini'))
  trainci = check_ci(ev.concordance_td())
  print(trainci)
  #testing c-index
  surv = model.predict_surv_df(x_test)
  ev = EvalSurv(surv, durations_test, events_test, censor_surv='km')
  # testci = check_ci(1- ev.concordance_td('antolini'))
  testci = check_ci(ev.concordance_td())
  print(testci)

  resultdf.append((ind,i,lw,en,es,ns,
                    trainci,
                  testci))


  
  cols=['seed','rank','lossweight','encodeNum','endoeSize','netSize','trainCI','testCI']

  results = pd.DataFrame(resultdf,columns = cols)
  avgtestci = results['testCI'].mean()
  avgtrainci = results['trainCI'].mean()
  print( "avgTRAIN"+ str(avgtrainci))
  print("avgTEST" + str(avgtestci))

  #avglist = []

resultname = result_folder + '11.21.auto_deepsurv' + 'final17' + ".csv"
results.to_csv(resultname,index=False,header = True)
    
#cols=['seed','rank','trainCI','testCI']

#cols1=['run','rank','trainCI','testCI']

#results = pd.DataFrame(resultdf,columns = cols)
#avgtestci = results['testCI'].mean()
#avgtrainci = results['trainCI'].mean()
#resultmean = pd.DataFrame(resultmeandf,columns = cols1)
#print(avgtestci)
#print(avgtrainci)
#resultname = result_folder + 'auto_deepsurv' + 'ranklisttopcox' +'simplenet' + ".csv"
#results.to_csv(resultname,index=False,header = True)


seed:1
rank: 17
Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 'age',
       'stage'],
      dtype='object')
0:	[0s / 0s],		train_loss: 1.4251,	train_loss_surv: 4.6970,	train_loss_ae: 0.0228,	val_loss: 0.5974,	val_loss_surv: 1.9581,	val_loss_ae: 0.0142
1:	[0s / 0s],		train_loss: 0.3183,	train_loss_surv: 1.0258,	train_loss_ae: 0.0151,	val_loss: 0.3115,	val_loss_surv: 1.0061,	val_loss_ae: 0.0138
2:	[0s / 0s],		train_loss: 0.2337,	train_loss_surv: 0.7435,	train_loss_ae: 0.0153,	val_loss: 0.3075,	val_loss_surv: 0.9926,	val_loss_ae: 0.0139
3:	[0s / 0s],		train_loss: 0.2617,	train_loss_surv: 0.8372,	train_loss_ae: 0.0151,	val_loss: 0.2869,	val_loss_surv: 0.9238,	val_loss_ae: 0.0139
4:	[0s / 0s],		train_loss: 0.2353,	train_loss_surv: 0.7489,	train_loss_ae: 0.0153,	val_loss: 0.2792,	val_loss_surv: 0.9000,	val_loss_ae: 0.0131
5:	[0s / 1s],		train_loss: 0.2831,	train_loss_surv: 0.9122,	train_loss_ae: 0.0135,	val_loss: 0.2735,	val_loss_surv: 0.8842,	val_loss_ae: 0.0118
6:	[0s / 

In [ ]:
out_features

100

In [ ]:

resultmeandf = []
for encode in encodef:
  resultdf = []
  for ind in np.arange(1,21): #go through all 20 train/test pairs
    torch.manual_seed(0)
    print('encodeNum:' + str(encode))
    print('start with seed' + str(ind))
    i = 17
    print('current rank: ' + str(i))

    df_train = pd.read_csv(fac_folder + 'train' + str(ind) + "rank" + str(i) + ".csv",header = None)
    df_test = pd.read_csv(fac_folder + 'test' + str(ind) + "rank"+ str(i) + ".csv",header= None)
    testind = np.array(pd.read_csv(ind_folder + 'testind' + str(ind)+ ".csv")["x"]) - 1 # minus 1 to match the array index in Python
    trainind = np.array(pd.read_csv(ind_folder + 'trainind' + str(ind)+ ".csv")["x"]) - 1

    df_train["age"] = Age[trainind]
    df_train["stage"] = Stage[trainind]
    df_test["age"] = Age[testind]
    df_test["stage"] = Stage[testind]

    Y_train = Y[trainind]
    Y_test = Y[testind]
    E_train = E[trainind]
    E_test = E[testind]

    df_train["time"] = Y_train.astype('float32')
    df_train["event"] = E_train.astype('float32')
    df_train,df_val = train_test_split(df_train, test_size=0.2)
    df_test["time"] = Y_test.astype('float32')
    df_test["event"] = E_test.astype('float32')

    #cols_standardize = ['x0', 'x1', 'x2', 'x3', 'x8']
    var_num = i + 2
    cols_leave = df_train.columns[0:var_num]
    print(cols_leave)
    #standardize = [([col], StandardScaler()) for col in cols_standardize]
    leave = [(col, None) for col in cols_leave]
    x_mapper = DataFrameMapper(leave)

    x_train = x_mapper.fit_transform(df_train).astype('float32')
    x_val = x_mapper.transform(df_val).astype('float32')
    x_test = x_mapper.transform(df_test).astype('float32')


    num_durations = 100
    labtrans = LogisticHazard.label_transform(num_durations)
    get_target = lambda df: (df['time'].values, df['event'].values)
    y_train_surv = labtrans.fit_transform(*get_target(df_train))
    y_val_surv = labtrans.transform(*get_target(df_val))

    train = tt.tuplefy(x_train, (y_train_surv, x_train))
    val = tt.tuplefy(x_val, (y_val_surv, x_val))

    # We don't need to transform the test labels
    durations_test, events_test = get_target(df_test)
    durations_train, events_train = get_target(df_train)

    class NetAESurv(nn.Module):
      def __init__(self, in_features, encoded_features, out_features):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(in_features, 512), nn.ReLU(),
            nn.Linear(512, 128), nn.ReLU(),
    
            nn.Linear(128, encoded_features),
        )
        self.decoder = nn.Sequential(
            nn.Linear(encoded_features, 128), nn.ReLU(),
    
            nn.Linear(128, 512), nn.ReLU(),
            nn.Linear(512, in_features),
        )
        self.surv_net = nn.Sequential(
            nn.Linear(encoded_features, 128), nn.ReLU(),
            nn.Linear(128, 64), nn.ReLU(),
            nn.Linear(64, out_features),
        )

      def forward(self, input):
        encoded = self.encoder(input)
        decoded = self.decoder(encoded)
        phi = self.surv_net(encoded)
        return phi, decoded

      def predict(self, input):
        # Will be used by model.predict later.
        # As this only has the survival output, 
        # we don't have to change LogisticHazard.
        encoded = self.encoder(input)
        return self.surv_net(encoded)

    in_features = x_train.shape[1]
    encoded_features = encode
    out_features = labtrans.out_features
    net = NetAESurv(in_features, encoded_features, out_features)

    class LossAELogHaz(nn.Module):
      def __init__(self, alpha):
        super().__init__()
        assert (alpha >= 0) and (alpha <= 1), 'Need `alpha` in [0, 1].'
        self.alpha = alpha
        self.loss_surv = NLLLogistiHazardLoss()
        self.loss_ae = nn.MSELoss()
        
      def forward(self, phi, decoded, target_loghaz, target_ae):
        idx_durations, events = target_loghaz
        loss_surv = self.loss_surv(phi, idx_durations, events)
        loss_ae = self.loss_ae(decoded, target_ae)
        return self.alpha * loss_surv + (1 - self.alpha) * loss_ae

    loss = LossAELogHaz(0.5)
    model = LogisticHazard(net, tt.optim.Adam(0.01), duration_index=labtrans.cuts, loss=loss)

    metrics = dict(
    loss_surv = LossAELogHaz(1),
    loss_ae   = LossAELogHaz(0)
    )
    callbacks = [tt.cb.EarlyStopping()]

    batch_size = 16
    epochs = 200
    model.fit(*train, batch_size, epochs, callbacks, val_data=val, metrics=metrics)

    def check_ci(ci):
      if ci < 0.5:
        ci = 1 - ci
      else:
          ci = ci
      return ci
    #training c-index
    surv = model.predict_surv_df(x_train)
    ev = EvalSurv(surv, durations_train, events_train, censor_surv='km')
    # trainci = check_ci(1- ev.concordance_td('antolini'))
    trainci = check_ci(ev.concordance_td())
    print(trainci)
    #testing c-index
    surv = model.predict_surv_df(x_test)
    ev = EvalSurv(surv, durations_test, events_test, censor_surv='km')
    # testci = check_ci(1- ev.concordance_td('antolini'))
    testci = check_ci(ev.concordance_td())
    print(testci)

    resultdf.append((ind,i,
                      trainci,
                    testci))
    
  cols=['seed','rank','trainCI','testCI']
  results = pd.DataFrame(resultdf,columns = cols)
  avgtestci = results['testCI'].mean()
  avgtrainci = results['trainCI'].mean()


  resultmeandf.append((encode,i,
                    avgtrainci,
                  avgtestci))
cols1=['encode','rank','trainCI','testCI']

results = pd.DataFrame(resultdf,columns = cols)
avgtestci = results['testCI'].mean()
avgtrainci = results['trainCI'].mean()
resultmean = pd.DataFrame(resultmeandf,columns = cols1)
print(avgtestci)

resultname = result_folder + 'auto_deepsurv' + 'encodenum10-90-1' + ".csv"
resultmean.to_csv(resultname,index=False,header = True)


Streaming output truncated to the last 5000 lines.
1:	[0s / 0s],		train_loss: 0.4830,	train_loss_surv: 0.9473,	train_loss_ae: 0.0186,	val_loss: 0.4533,	val_loss_surv: 0.8934,	val_loss_ae: 0.0131
2:	[0s / 0s],		train_loss: 0.3819,	train_loss_surv: 0.7485,	train_loss_ae: 0.0153,	val_loss: 0.4763,	val_loss_surv: 0.9374,	val_loss_ae: 0.0151
3:	[0s / 0s],		train_loss: 0.3952,	train_loss_surv: 0.7772,	train_loss_ae: 0.0132,	val_loss: 0.4408,	val_loss_surv: 0.8714,	val_loss_ae: 0.0102
4:	[0s / 0s],		train_loss: 0.3866,	train_loss_surv: 0.7643,	train_loss_ae: 0.0089,	val_loss: 0.4528,	val_loss_surv: 0.8995,	val_loss_ae: 0.0061
5:	[0s / 1s],		train_loss: 0.3705,	train_loss_surv: 0.7363,	train_loss_ae: 0.0047,	val_loss: 0.4392,	val_loss_surv: 0.8735,	val_loss_ae: 0.0049
6:	[0s / 1s],		train_loss: 0.3736,	train_loss_surv: 0.7416,	train_loss_ae: 0.0057,	val_loss: 0.4637,	val_loss_surv: 0.9195,	val_loss_ae: 0.0079
7:	[0s / 1s],		train_loss: 0.3675,	train_loss_surv: 0.7320,	train_loss_ae: 0.0030,	va